# Entregable 2
## Inteligencia Artificial

En este entregable se realizará la aplicación de Regresión Lineal y Regresión logística. 

## Regresión Lineal:

Trabajaremos con los datos de la clase de EDA [(Clase02)](https://github.com/andrescastro-itm/AI_ITM/blob/main/Clase02/EDA.ipynb). 

1. Deben cargar los datos y realizar una depuración de los mismos (pueden usar lo que se hizo en la clase de EDA)

2. Realizar una división del conjunto de entrenamiento en conjuntos de entrenamiento/prueba, con los porcentajes que consideren

3. Posteriormente, deben entrenar un regresor lineal y reportar el error cuadrático medio en la predicción del precio de venta _(SalePrice)_ en el conjunto de prueba

4. Realizar una remoción de algunas características (a discreción de ustedes) y realizar de nuevo el entrenamiento del regresor lineal. Reportar el error cuadrático medio en la predicción del precio de venta _(SalePrice)_ en el conjunto de prueba.

In [1]:
#celda para importar, pueden agregar las que requieran

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Celda para la carga de datos:

In [2]:
df = pd.read_csv("train.csv",index_col=0)

## Solución a punto 1.

## Solución a punto 2.

## Solución a punto 3.

## Regresión Logística:

Estos puntos los deben trabajar con **alguno (o si prefieren, todos)** de los conjuntos de datos del proyecto final. 

Lo pueden realizar de dos formas:

- Seleccionar únicamente los datos correspondientes a las clases 1 y 2, es decir, los datos con etiqueta Sabana o Bosque. Verificar que les deben quedar en total 29540 datos entre los conjuntos de entrenamiento, validación y prueba. ó
- Trabajar con las tres clases aprovechando que sklearn utiliza el esquema uno-contra-todos (one-vs-rest OvR)

Realizar las siguientes tareas:

1. Entrenar un modelo de regresión logística con el conjunto de entrenamiento
2. Reportar los resultados logrados con el conjunto de validación en términos de precisión, recall y F1-score. Para este punto pueden usar [classification_report de sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html#sklearn.metrics.classification_report)
3. Mostrar una matriz de confusión para la clasificación hecha en el conjunto de validación


## Solución a punto 1.

## Solución a punto 2.

## Solución a punto 3.